In [52]:
import pandas as pd # version 2.2.3
import yfinance as yf # version 0.2.49
import numpy as np # version 2.1.3
from datetime import datetime, timedelta 
import pytz # version 2024.2
import analysis_functs as funct
import matplotlib.pyplot as plt # version 3.10.0

# Get current date 
current_date = datetime.now(pytz.timezone('America/Chicago'))

# Analysis dates
end_date = (current_date-timedelta(days=1)).strftime("%Y-%m-%d") # yyyy-mm-dd
start_date = (current_date-timedelta(days=1)).replace(year=current_date.year-1).strftime("%Y-%m-%d") # accounts for leap years and also for yfinance issues with using day-of data
# currently set to 1 year befored

# Frequency
freq = '1mo'
# 5m for 5 minutes, 1d for day, 1wk for week, 1mo for month, etc.

# start_date = "2023-11-16" # hard coded dates for analysis
# end_date = "2024-11-16"

# Load original portfolio
portf = pd.read_csv("portfolios/crypto.csv")

# Calculate metrics for original portfolio
original_metrics = funct.calculate_portfolio_metrics(portf, start_date, end_date)

# # Download S&P 500 (or another market index) data
market_data = yf.download('^GSPC', start=start_date, end=end_date, interval=freq)['Close']

# Calculate market returns at specified frequency (monthly right now)
market_returns = market_data.pct_change().dropna()

# Calculate expected return value for the market
market_ret = funct.calc_exp_ret(market_data)

# Fetch risk-free rate (10-Year Treasury Yield)
risk_free_data = yf.download('^TNX', start=start_date, end=end_date, interval=freq)
risk_free_rate = risk_free_data['Close'].dropna().iloc[-1].item() / 100

# Calculate Beta
original_beta = funct.calculate_beta(
    original_metrics[2], market_returns
    )

# Calculate Sharpe
original_sharpe = funct.calc_sharpe_ratio(
    original_metrics[0], risk_free_rate, original_metrics[1]
    )

# Calculate alpha for portfolio
original_alpha = float(funct.calculate_alpha(original_metrics[0],risk_free_rate,original_beta,market_ret).iloc[0])

# Display comparison
print("Original Portfolio:")
print(f"Expected Return: {original_metrics[0]:.4f}")
print(f"Portfolio Std Dev: {original_metrics[1]:.4f}")
print(f"Beta: {original_beta:.4f}")
print(f"Sharpe Ratio: {original_sharpe:.4f}")
print(f"Alpha: {original_alpha:.4f}")

[*********************100%***********************]  3 of 3 completed
[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed

Original Portfolio:
Expected Return: 0.8437
Portfolio Std Dev: 0.7290
Beta: 1.3233
Sharpe Ratio: 1.0946
Alpha: 0.7040


In [42]:

# Primary function, gets data and calcs preliminary metrics for the a given portfolio.
def calculate_portfolio_metrics(portf, start_date, end_date):
    print("hi")
    # Separate cash and stock rows
    cash_row = portf[portf['TICKER'] == 'USD']
    stock_rows = portf[portf['TICKER'] != 'USD'].copy()  # Ensure this is a copy

    # Fetch data for the tickers in the portfolio
    tickers = stock_rows['TICKER'].tolist()
    data = yf.download(tickers, start=start_date, end=end_date, interval="1mo")['Close']

    # Calculate monthly returns
    returns = data.pct_change().dropna()

    # Calculate total investment in each stock
    current_prices = data.iloc[-1]  # Last row gives the latest prices
    stock_rows['Investment'] = stock_rows['QUANTITY'].values * current_prices.values

    # Check if cash row is empty and handle accordingly
    if not cash_row.empty:
        cash_quantity = cash_row['QUANTITY'].iloc[0]
    else:
        cash_quantity = 0

    # Add the cash row back for total investment
    total_investment = stock_rows['Investment'].sum() + cash_quantity

    # Calculate weights
    stock_rows['Weight'] = stock_rows['Investment'] / total_investment

    # Handle cash weight separately
    cash_weight = cash_quantity / total_investment if cash_quantity > 0 else 0

    # Calculate weighted monthly returns for the portfolio
    weighted_returns = (returns * stock_rows.set_index('TICKER')['Weight']).sum(axis=1)

    # Adjust weighted returns to include cash
    weighted_returns = weighted_returns * (1 - cash_weight)

    # Expected annual return
    # expected_return = weighted_returns.mean() * 12 # using arithmetric mean
    expected_return = (1+weighted_returns).prod()**(12/weighted_returns.size) - 1 # using geometric mean

    # Portfolio variance and standard deviation (risk)
    portfolio_variance = np.dot(stock_rows.set_index('TICKER')['Weight'].T, 
                                np.dot(returns.cov() * 12, 
                                       stock_rows.set_index('TICKER')['Weight']))
    portfolio_variance *= (1 - cash_weight)**2  # Adjust for cash weight
    portfolio_std_dev = np.sqrt(portfolio_variance)

    return expected_return, portfolio_std_dev, weighted_returns, cash_weight, data, stock_rows


In [48]:

# Get current date 
current_date = datetime.now(pytz.timezone('America/Chicago'))

# Analysis dates
end_date = (current_date-timedelta(days=1)).strftime("%Y-%m-%d") # yyyy-mm-dd
start_date = (current_date-timedelta(days=1)).replace(year=current_date.year-1).strftime("%Y-%m-%d") # accounts for leap years and also for yfinance issues with using day-of data
# currently set to 1 year befored

# Frequency
freq = '1mo'
# 5m for 5 minutes, 1d for day, 1wk for week, 1mo for month, etc.

# start_date = "2023-11-16" # hard coded dates for analysis
# end_date = "2024-11-16"

# Load original portfolio
portf = pd.read_csv("portfolios/crypto.csv")

# Calculate metrics for original portfolio
original_metrics = calculate_portfolio_metrics(portf, start_date, end_date)

# # Download S&P 500 (or another market index) data
market_data = yf.download('^GSPC', start=start_date, end=end_date, interval=freq)['Close']

# Calculate market returns at specified frequency (monthly right now)
market_returns = market_data.pct_change().dropna()

# Calculate expected return value for the market
market_ret = funct.calc_exp_ret(market_data)

# Fetch risk-free rate (10-Year Treasury Yield)
risk_free_data = yf.download('^TNX', start=start_date, end=end_date, interval=freq)
risk_free_rate = risk_free_data['Close'].dropna().iloc[-1].item() / 100

# Calculate Beta
original_beta = funct.calculate_beta(
    original_metrics[2], market_returns
    )

# Calculate Sharpe
original_sharpe = funct.calc_sharpe_ratio(
    original_metrics[0], risk_free_rate, original_metrics[1]
    )

# Calculate alpha for portfolio
original_alpha = float(funct.calculate_alpha(original_metrics[0],risk_free_rate,original_beta,market_ret).iloc[0])

# Display comparison
print("Original Portfolio:")
print(f"Expected Return: {original_metrics[0]:.4f}")
print(f"Portfolio Std Dev: {original_metrics[1]:.4f}")
print(f"Beta: {original_beta:.4f}")
print(f"Sharpe Ratio: {original_sharpe:.4f}")
print(f"Alpha: {original_alpha:.4f}")

[*********************100%***********************]  3 of 3 completed


[*********************100%***********************]  1 of 1 completed

hi



[*********************100%***********************]  1 of 1 completed

Original Portfolio:
Expected Return: 0.8437
Portfolio Std Dev: 0.7290
Beta: 1.3233
Sharpe Ratio: 1.0946
Alpha: 0.7040


In [49]:
portfolio_variance

np.float64(0.5314252573490492)

In [50]:
# ef calculate_portfolio_metrics(portf, start_date, end_date):

# Separate cash and stock rows
cash_row = portf[portf['TICKER'] == 'USD']
stock_rows = portf[portf['TICKER'] != 'USD'].copy()  # Ensure this is a copy

# Fetch data for the tickers in the portfolio
tickers = stock_rows['TICKER'].tolist()
data = yf.download(tickers, start=start_date, end=end_date, interval="1mo")['Close']

# Calculate monthly returns
returns = data.pct_change().dropna()

# Calculate total investment in each stock
current_prices = data.iloc[-1]  # Last row gives the latest prices
stock_rows['Investment'] = stock_rows['QUANTITY'].values * current_prices.values

# Check if cash row is empty and handle accordingly
if not cash_row.empty:
    cash_quantity = cash_row['QUANTITY'].iloc[0]
else:
    cash_quantity = 0

# Add the cash row back for total investment
total_investment = stock_rows['Investment'].sum() + cash_quantity

# Calculate weights
stock_rows['Weight'] = stock_rows['Investment'] / total_investment

# Handle cash weight separately
cash_weight = cash_quantity / total_investment if cash_quantity > 0 else 0

# Calculate weighted monthly returns for the portfolio
weighted_returns = (returns * stock_rows.set_index('TICKER')['Weight']).sum(axis=1)

# Adjust weighted returns to include cash
weighted_returns = weighted_returns * (1 - cash_weight)

# Expected annual return
# expected_return = weighted_returns.mean() * 12 # using arithmetric mean
expected_return = (1+weighted_returns).prod()**(12/weighted_returns.size) - 1 # using geometric mean

# Portfolio variance and standard deviation (risk)
portfolio_variance = np.dot(stock_rows.set_index('TICKER')['Weight'].T, 
                            np.dot(returns.cov() * 12, 
                                    stock_rows.set_index('TICKER')['Weight']))
# portfolio_variance *= (1 - cash_weight)**2  # Adjust for cash weight
# portfolio_std_dev = np.sqrt(portfolio_variance)

[*********************100%***********************]  3 of 3 completed

In [51]:
stock_rows

,TICKER,QUANTITY,Investment,Weight
0,BTC-USD,0.012137,1005.670379,0.098577
1,XMR-USD,4.790000,9196.057410,0.901412
2,ETH-USD,0.000530,0.112428,0.000011


In [46]:
total_investment

np.float64(10201.840216546902)

In [8]:
portfolio_variance

np.float64(0.017598685373148886)

In [10]:
portfolio_variance

np.float64(0.018900217588806955)

In [12]:
returns

Ticker,AAPL,ADI,AMAT,AMP,AMR,BRK-B,CE,ELV,GOOG,JBL,JPM,MRK,PG,PWR,REGN,SCHW,STLA,XOM,YUM
Date,,,,,,,,,,,,,,,,,,,
2024-05-01,0.128691,0.168885,0.082708,0.060273,-0.035797,0.044539,-0.005698,0.018729,0.056608,0.013122,0.062960,-0.028479,0.014721,0.067583,0.100487,-0.009060,0.067059,-0.008540,-0.027044
2024-06-01,0.097038,-0.026568,0.099220,-0.018086,-0.110586,-0.018340,-0.112799,0.006277,0.054380,-0.084408,-0.001826,-0.013860,0.002310,-0.079184,0.072305,0.008896,-0.108666,-0.010276,-0.031427
2024-07-01,0.054411,0.017701,-0.100809,0.006742,0.053042,0.077925,0.046408,-0.015163,-0.054917,0.035665,0.052111,-0.080721,-0.025224,0.044433,0.026793,-0.115348,-0.159194,0.030142,0.002793
2024-08-01,0.031160,0.014954,-0.070405,0.045039,-0.190447,0.085336,-0.070065,0.046727,-0.046434,-0.030088,0.062245,0.047025,0.073474,0.037107,0.097759,-0.001381,0.005392,-0.005481,0.015735
2024-09-01,0.018645,-0.019886,0.026246,0.049180,-0.012419,-0.032905,0.041041,-0.066243,0.012598,0.097391,-0.062011,-0.041283,0.009677,0.083669,-0.112645,-0.000474,-0.162694,0.001897,0.040669
2024-10-01,-0.030429,-0.026861,-0.101312,0.086184,-0.118046,-0.020293,-0.073478,-0.217346,0.034256,0.027205,0.052452,-0.092950,-0.046305,0.011672,-0.202656,0.092887,-0.030605,-0.003754,-0.061198
2024-11-01,0.050551,-0.022679,-0.037834,0.124770,0.178925,0.071188,-0.415606,0.002957,-0.012740,0.103502,0.132166,-0.006646,0.091621,0.142539,-0.104963,0.168431,-0.030837,0.010104,0.059317
2024-12-01,0.056316,-0.025636,-0.066954,-0.069658,-0.185080,-0.061571,-0.054637,-0.093523,0.117016,0.060070,-0.040085,-0.021251,-0.064766,-0.082637,-0.050505,-0.102661,-0.011364,-0.080587,-0.034403
2025-01-01,-0.057583,0.001540,0.108959,0.020529,-0.084699,0.033953,0.026441,0.077036,0.080830,0.128631,0.115097,0.001317,-0.009902,-0.026705,-0.055241,0.117687,0.006130,-0.006879,-0.022568


In [17]:
returns.sum()

Ticker
AAPL     0.315202
ADI      0.107216
AMAT    -0.232650
AMP      0.190327
AMR     -0.761310
BRK-B    0.243631
CE      -0.836338
ELV     -0.205641
GOOG     0.053637
JBL      0.166145
JPM      0.245770
MRK     -0.271028
PG       0.113205
PWR     -0.045510
REGN    -0.123059
SCHW     0.019150
STLA    -0.444211
XOM     -0.018046
YUM      0.155888
dtype: float64

In [21]:
returns.sum().cov()

TypeError: Series.cov() missing 1 required positional argument: 'other'

In [19]:
returns.corr()

Ticker,AAPL,ADI,AMAT,AMP,AMR,BRK-B,CE,ELV,GOOG,JBL,JPM,MRK,PG,PWR,REGN,SCHW,STLA,XOM,YUM
Ticker,,,,,,,,,,,,,,,,,,,
AAPL,1.000000,0.543785,0.210102,0.191929,0.062399,0.188323,-0.233908,0.074082,0.099161,-0.022353,0.169610,-0.124497,0.121047,0.233699,0.404838,-0.164761,-0.087530,-0.179649,-0.033120
ADI,0.543785,1.000000,0.145092,0.244650,-0.183951,0.537105,-0.063706,0.253593,-0.203947,0.019203,0.303607,-0.215747,0.204318,0.070531,0.441765,-0.078659,0.369292,0.206600,0.249571
AMAT,0.210102,0.145092,1.000000,0.079831,0.164119,-0.223333,0.261167,0.415403,0.598804,0.183833,0.159053,0.322532,-0.033190,0.148949,0.165201,0.324992,0.091854,-0.221935,-0.394177
AMP,0.191929,0.244650,0.079831,1.000000,0.380844,0.423357,-0.422460,-0.189865,-0.019625,0.571976,0.754826,-0.244849,0.381962,0.788940,-0.444343,0.768923,-0.032473,0.222329,-0.017341
AMR,0.062399,-0.183951,0.164119,0.380844,1.000000,0.078621,-0.096494,0.115735,0.076755,0.319165,0.253451,0.018924,0.217975,0.713363,-0.218045,0.292968,-0.237234,0.261781,-0.133304
BRK-B,0.188323,0.537105,-0.223333,0.423357,0.078621,1.000000,-0.431270,0.487530,-0.665966,0.063945,0.621263,-0.024939,0.658061,0.221618,0.340921,0.188131,0.100631,0.633324,0.537976
CE,-0.233908,-0.063706,0.261167,-0.422460,-0.096494,-0.431270,1.000000,-0.000223,0.359723,-0.088437,-0.401007,0.061994,-0.602611,-0.074127,0.166835,-0.492257,-0.099609,-0.194517,-0.545522
ELV,0.074082,0.253593,0.415403,-0.189865,0.115735,0.487530,-0.000223,1.000000,-0.210708,-0.168888,0.154550,0.657999,0.557517,-0.058541,0.711711,-0.031822,0.236873,0.276017,0.282613
GOOG,0.099161,-0.203947,0.598804,-0.019625,0.076755,-0.665966,0.359723,-0.210708,1.000000,0.373768,0.073655,0.152007,-0.583957,0.156985,-0.272385,0.189850,0.172978,-0.820142,-0.835663


In [11]:
returns.cov()

Ticker,AAPL,ADI,AMAT,AMP,AMR,BRK-B,CE,ELV,GOOG,JBL,JPM,MRK,PG,PWR,REGN,SCHW,STLA,XOM,YUM
Ticker,,,,,,,,,,,,,,,,,,,
AAPL,0.003521,0.002090,0.001067,0.000758,0.000453,0.000633,-0.002077,0.000358,0.000455,-0.000111,0.000773,-0.000325,0.000345,0.001248,0.002409,-0.000910,-0.000379,-0.000332,-0.000139
ADI,0.002090,0.004196,0.000805,0.001054,-0.001457,0.001972,-0.000618,0.001338,-0.001022,0.000104,0.001511,-0.000615,0.000636,0.000411,0.002870,-0.000474,0.001748,0.000417,0.001142
AMAT,0.001067,0.000805,0.007328,0.000455,0.001718,-0.001084,0.003346,0.002896,0.003964,0.001312,0.001046,0.001215,-0.000137,0.001148,0.001418,0.002588,0.000574,-0.000592,-0.002384
AMP,0.000758,0.001054,0.000455,0.004426,0.003099,0.001597,-0.004207,-0.001029,-0.000101,0.003171,0.003858,-0.000717,0.001221,0.004725,-0.002965,0.004760,-0.000158,0.000461,-0.000082
AMR,0.000453,-0.001457,0.001718,0.003099,0.014960,0.000545,-0.001766,0.001153,0.000726,0.003253,0.002382,0.000102,0.001281,0.007854,-0.002675,0.003334,-0.002120,0.000997,-0.001152
BRK-B,0.000633,0.001972,-0.001084,0.001597,0.000545,0.003213,-0.003659,0.002250,-0.002919,0.000302,0.002705,-0.000062,0.001792,0.001131,0.001938,0.000992,0.000417,0.001118,0.002155
CE,-0.002077,-0.000618,0.003346,-0.004207,-0.001766,-0.003659,0.022401,-0.000003,0.004164,-0.001103,-0.004611,0.000408,-0.004334,-0.000999,0.002504,-0.006855,-0.001089,-0.000907,-0.005769
ELV,0.000358,0.001338,0.002896,-0.001029,0.001153,0.002250,-0.000003,0.006630,-0.001327,-0.001146,0.000967,0.002357,0.002182,-0.000429,0.005813,-0.000241,0.001409,0.000700,0.001626
GOOG,0.000455,-0.001022,0.003964,-0.000101,0.000726,-0.002919,0.004164,-0.001327,0.005981,0.002409,0.000438,0.000517,-0.002170,0.001093,-0.002113,0.001366,0.000977,-0.001975,-0.004566


In [15]:
data.pct_change()

Ticker,AAPL,ADI,AMAT,AMP,AMR,BRK-B,CE,ELV,GOOG,JBL,JPM,MRK,PG,PWR,REGN,SCHW,STLA,XOM,YUM
Date,,,,,,,,,,,,,,,,,,,
2024-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-01,0.128691,0.168885,0.082708,0.060273,-0.035797,0.044539,-0.005698,0.018729,0.056608,0.013122,0.062960,-0.028479,0.014721,0.067583,0.100487,-0.009060,0.067059,-0.008540,-0.027044
2024-06-01,0.097038,-0.026568,0.099220,-0.018086,-0.110586,-0.018340,-0.112799,0.006277,0.054380,-0.084408,-0.001826,-0.013860,0.002310,-0.079184,0.072305,0.008896,-0.108666,-0.010276,-0.031427
2024-07-01,0.054411,0.017701,-0.100809,0.006742,0.053042,0.077925,0.046408,-0.015163,-0.054917,0.035665,0.052111,-0.080721,-0.025224,0.044433,0.026793,-0.115348,-0.159194,0.030142,0.002793
2024-08-01,0.031160,0.014954,-0.070405,0.045039,-0.190447,0.085336,-0.070065,0.046727,-0.046434,-0.030088,0.062245,0.047025,0.073474,0.037107,0.097759,-0.001381,0.005392,-0.005481,0.015735
2024-09-01,0.018645,-0.019886,0.026246,0.049180,-0.012419,-0.032905,0.041041,-0.066243,0.012598,0.097391,-0.062011,-0.041283,0.009677,0.083669,-0.112645,-0.000474,-0.162694,0.001897,0.040669
2024-10-01,-0.030429,-0.026861,-0.101312,0.086184,-0.118046,-0.020293,-0.073478,-0.217346,0.034256,0.027205,0.052452,-0.092950,-0.046305,0.011672,-0.202656,0.092887,-0.030605,-0.003754,-0.061198
2024-11-01,0.050551,-0.022679,-0.037834,0.124770,0.178925,0.071188,-0.415606,0.002957,-0.012740,0.103502,0.132166,-0.006646,0.091621,0.142539,-0.104963,0.168431,-0.030837,0.010104,0.059317
2024-12-01,0.056316,-0.025636,-0.066954,-0.069658,-0.185080,-0.061571,-0.054637,-0.093523,0.117016,0.060070,-0.040085,-0.021251,-0.064766,-0.082637,-0.050505,-0.102661,-0.011364,-0.080587,-0.034403
